In [ ]:
%run params.ipynb
import rules.example

for name, rules in [('example', rules.example.build_rules())]:
    PARAMS['save_file'] = True
    PARAMS['name'] = name
    PARAMS['graph'] = rules['graph']          
    PARAMS['rules'] = rules['rules']        
    
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [ ]:
import re
import rules.v01_oscillating as rul
from rules.enzymes import *

base_name = re.sub(".+\.", "", rul.__name__)

all_nodes = {
    Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, Dia1, Dia2, Ena_Vasp,
    IRSp53, LIMK, PAK, PIP2, PKD, Profilin, ROCK, Rac1, RhoA, SSH, Thymosin,
    WASP, WAVE
}

rules_map = {}
orig_rules = rul.build_rules()
rules_map[base_name] = orig_rules

# knockdown nodes
for node in all_nodes:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']

    graph.remove_edges_from([(node, target) for target in list(graph[node])])
    rules = re.sub('1:\s*' + node + '\*\s*=.+', '', rules)
    rules = rules + '1: ' + node + '* = False \n'

    rules_map[base_name + "_del_" + node] = {"graph": graph, "rules": rules}

# multi-knockdowns
for nodes in [[Cdc42, Rac1], [Dia1, Dia2]]:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']

    for node in nodes:
        graph.remove_edges_from([(node, target)
                                 for target in list(graph[node])])
        rules = re.sub('1:\s*' + node + '\*\s*=.+', '', rules)
        rules = rules + '1: ' + node + '* = False \n'

    rules_map[base_name + "_del_" + "_".join(nodes)] = {
        "graph": graph,
        "rules": rules
    }

sorted(rules_map.keys())

# print(rules_map['v01_oscillating_del_Cdc42']['rules'])

In [5]:
import re
import rules.v02_fixing_Arp2_3 as rul
from rules.enzymes import *
import itertools

base_name = re.sub(".+\.", "", rul.__name__)

all_nodes = {
    Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, Dia1, Dia2, Ena_Vasp,
    IRSp53, LIMK, PAK, PIP2, PKD, Profilin, ROCK, Rac1, RhoA, SSH, Thymosin,
    WASP, WAVE
}

rules_map = {}
orig_rules = rul.build_rules()
rules_map[base_name] = orig_rules

# knockdowns
for nodes in sorted([{a, b} for a, b in itertools.combinations_with_replacement(sorted(all_nodes), 2)]):
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']

    for node in nodes:
        graph.remove_edges_from([(node, target)
                                 for target in list(graph[node])])
        rules = re.sub('1:\s*' + node + '\*\s*=.+', '', rules)
        rules = rules + '1: ' + node + '* = False \n'

    rules_map[base_name + "_del_" + "_".join(nodes)] = {
        "graph": graph,
        "rules": rules
    }

sorted(rules_map.keys())

# print(rules_map['v01_oscillating_del_Cdc42']['rules'])

['v02_fixing_Arp2_3',
 'v02_fixing_Arp2_3_del_Arp2_3',
 'v02_fixing_Arp2_3_del_Arp2_3_Cortactin',
 'v02_fixing_Arp2_3_del_Arp2_3_LIMK',
 'v02_fixing_Arp2_3_del_Arp2_3_Thymosin',
 'v02_fixing_Arp2_3_del_Arp2_3_WASP',
 'v02_fixing_Arp2_3_del_Arp2_3_WAVE',
 'v02_fixing_Arp2_3_del_CP',
 'v02_fixing_Arp2_3_del_CP_Arp2_3',
 'v02_fixing_Arp2_3_del_CP_Cdc42',
 'v02_fixing_Arp2_3_del_CP_Cofilin',
 'v02_fixing_Arp2_3_del_CP_Coronin',
 'v02_fixing_Arp2_3_del_CP_Cortactin',
 'v02_fixing_Arp2_3_del_CP_Dia1',
 'v02_fixing_Arp2_3_del_CP_Ena_Vasp',
 'v02_fixing_Arp2_3_del_CP_IRSp53',
 'v02_fixing_Arp2_3_del_CP_LIMK',
 'v02_fixing_Arp2_3_del_CP_PAK',
 'v02_fixing_Arp2_3_del_CP_PKD',
 'v02_fixing_Arp2_3_del_CP_Profilin',
 'v02_fixing_Arp2_3_del_CP_ROCK',
 'v02_fixing_Arp2_3_del_CP_Rac1',
 'v02_fixing_Arp2_3_del_CP_RhoA',
 'v02_fixing_Arp2_3_del_CP_SSH',
 'v02_fixing_Arp2_3_del_CP_Thymosin',
 'v02_fixing_Arp2_3_del_CP_WASP',
 'v02_fixing_Arp2_3_del_CP_WAVE',
 'v02_fixing_Arp2_3_del_Cdc42',
 'v02_fixing_A

In [ ]:
### run selected rules

%run params.ipynb

for name, rules in sorted(rules_map.items()):
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 64
    PARAMS['chunks'] = 16
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST]]
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [8]:
# build summary for selected rules map
import pandas as pd
import numpy as np
import os.path

def color_delta(src):
    base = src['base']    
    colors = [ 'red' if s > 0 else 'blue' if s < 0 else 'black' for s in np.sign(src - base)]    
    return [ 'color: {}'.format(c)  for c in colors]

cases = sorted(rules_map.keys())
rep = pd.read_csv("reports/"+cases[0]+"/weighted_activity.csv", index_col=0)
rep.columns = ["base"]
for case in cases[1:]:
    file_name = "reports/"+case+"/weighted_activity.csv"
    if os.path.isfile(file_name):
        name = re.sub(base_name+"_del_", "", case)
        rep[name] = pd.read_csv("reports/"+case+"/weighted_activity.csv", index_col=0).iloc[:, 0].rename(name)

rep['mean'] = rep.mean(axis=1)
rep.to_csv("reports/"+cases[0]+"_summary.csv")

styler = rep.style.apply(color_delta, axis=1)

with open('reports/'+cases[0]+'_summary.html', 'w') as fo:
    fo.write(styler.render())
    
styler

In [12]:
# build double knockdown summary for selected rules map
import pandas as pd
import numpy as np
import os.path

def color_delta(src, base):
    if src.name[1] == 'BR':
        b = base[0]
    else:
        b = base[1]
        
    colors = [ 'gray' if pd.isnull(s) else 'red' if s > 0 else 'blue' if s < 0 else 'black' for s in src - b]  
    return [ 'color: {}'.format(c)  for c in colors]


na = 'n/a'
BR = 'BR'
ST = 'ST'
midx = pd.MultiIndex.from_product([[na] + sorted(all_nodes), [BR, ST]])
report = pd.DataFrame(index=midx, columns=[na] + sorted(all_nodes), data=None)

cases = sorted(rules_map.keys())

#base
rep = pd.read_csv("reports/"+cases[0]+"/weighted_activity.csv", index_col=0)
base_BR = rep['activity, %']['Actin_BR']
base_ST = rep['activity, %']['Actin_ST']
report[na][na] = (base_BR, base_ST)

node_re = '|'.join(all_nodes)

for case in cases[1:]:
    file_name = "reports/"+case+"/weighted_activity.csv"   
    if os.path.isfile(file_name):               
        name = re.sub(base_name+"_del_", "", case)    
        match = re.match("("+node_re+")(_("+node_re+"))?", name) 
        a, b = match.group(1, 3)
        if b is None:
            b = na
        
        rep = pd.read_csv(file_name, index_col=0)
        values = (rep['activity, %']['Actin_BR'], rep['activity, %']['Actin_ST'])
        report[a][b] = values
        report[b][a] = values        


report.to_csv("reports/"+cases[0]+"_double_knockdown_summary.csv")       
styler = report.style.apply(color_delta, axis=1, base=(base_BR, base_ST))

with open('reports/'+cases[0]+'_double_knockdown_summary.html', 'w') as fo:
    fo.write(styler.render())

styler